# WORKING CAPITAL OPTIMIZATION

# BUSINESS UNDERSTANDING

Working capital optimization involves strategically managing a company's current assets and liabilities to enhance operational efficiency and financial health. It focuses on finding the right balance between accounts receivable (AR) and accounts payable (AP) to ensure sufficient liquidity while minimizing costs and risks

#### Problem Statement
This involves predicting the timing of customer payments and supplier payments to ensure cash flow and liquidity in a specific period for the company. By analyzing historical data and using predictive modeling techniques, the project aims to forecast the week customers are likely to pay the company and the week in which the company should pay suppliers.

These predictions will  enable the company to proactively manage cash flow, ensuring that customer payments are received on time to cover outgoing payments to suppliers. This helps maintain a healthy cash position, enhances liquidity, and allows the company to meet its financial obligations effectively

#### Aim
The project aims to optimize working capital management by leveraging accounts receivable and payable data. Through predictive analysis, the project aims to accurately forecast the timing of customer and supplier payments, enabling the company to ensure cash flow and liquidity within a specified perio

In [2]:
# importing libraries
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

# DATA UNDERSTANDING

##### Load the data

In [3]:
customers_df = pd.read_excel("Data/customers_data.xls")
payables_df = pd.read_excel("Data/payables_data.xls")
receivables_df = pd.read_excel("Data/receivables_data.xls")
suppliers_df= pd.read_excel("Data/suppliers_data.xls") 

##### Customers Data

In [4]:
customers_df.head()

,Customer ID,Customer Name,Customer Payment Terms,Address,Credit Limit
0,C0200769623-0,WAL-MAR corp,NAH4,55599 Katherine Harbors Suite 551\nWest Brenda...,50000
1,C0200980828-1,BEN E,NAD1,"5488 Michael Inlet\nElizabethport, MP 17624",50000
2,C0200792734-2,MDV/ trust,NAA8,"708 Taylor Cape\nJohnstad, MT 34743",100000
3,C0140105686-3,SYSC llc,CA10,"4113 Dana Ridges\nEast Clarencestad, IA 61466",100000
4,C0140106181-4,WAL-MAR foundation,NAH4,"2759 Kimberly Villages\nThompsonside, OR 79370",100000


In [5]:
customers_df.columns

Index(['Customer ID', 'Customer Name', 'Customer Payment Terms', 'Address',
       'Credit Limit'],
      dtype='object')

These columns represent:
* Customer ID: Unique identifier for each customer.    
* Customer Name: Name of the customer.    
* Customer Payment Terms: Terms and conditions for customer payments.   
* Address: Physical address or location of the customer.    
* Credit Limit: Maximum credit amount extended to the customer.   

In [6]:
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4977 entries, 0 to 4976
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Customer ID             4977 non-null   object
 1   Customer Name           4977 non-null   object
 2   Customer Payment Terms  4977 non-null   object
 3   Address                 4977 non-null   object
 4   Credit Limit            4977 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 194.5+ KB


There are no null values. All are objects except the Credit Limit. 

##### Receivables Data

In [7]:
receivables_df.head()

,Business Code,Customer Number,Customer Name,Payment_Date,Business Year,Posting_Date,Due_Date,Payterm,Invoice Currency,Total Open Amount,USD_CURRENNCY,Total Open Amount_USD,Customer Payment Terms,Invoice ID,Is Open,DUNNLEVEL,Credit_limit,Baseline_Date,Region
0,U001,200769623,WAL-MAR corp,2020-02-11 00:00:00,2020,2020-01-26,2020-02-10,15,USD,54273,USD,54273.280,NAH4,1.930438e+09,0,0,50000,2020-01-23,WEST
1,U001,200980828,BEN E,2019-08-08 00:00:00,2019,2019-07-22,2019-08-11,20,USD,79657,USD,79656.600,NAD1,1.929646e+09,0,2,50000,2019-07-20,MIDWEST
2,U001,200792734,MDV/ trust,2019-12-30 00:00:00,2019,2019-09-14,2019-09-29,15,USD,2254,USD,2253.860,NAA8,1.929874e+09,0,3,100000,2019-09-14,NORTHEAST
3,CA02,140105686,SYSC llc,01/00/1900,2020,2020-03-30,2020-04-10,11,CAD,3300,USD,2441.778,CA10,2.960623e+09,1,2,100000,2020-03-26,SOUTHWEST
4,U001,200769623,WAL-MAR foundation,2019-11-25 00:00:00,2019,2019-11-13,2019-11-28,15,USD,33133,USD,33133.290,NAH4,1.930148e+09,0,3,100000,2019-11-10,WEST


In [8]:
receivables_df.columns

Index(['Business Code', 'Customer Number', 'Customer Name', 'Payment_Date',
       'Business Year', 'Posting_Date', 'Due_Date', 'Payterm',
       'Invoice Currency', 'Total Open Amount', 'USD_CURRENNCY',
       'Total Open Amount_USD', 'Customer Payment Terms', 'Invoice ID',
       'Is Open', 'DUNNLEVEL', 'Credit_limit', 'Baseline_Date', 'Region'],
      dtype='object')

These columns represent:
* Business Code: Code representing the type of business transaction.
* Customer Number: Unique identifier for each customer.
* Customer Name: Name of the customer.
* Payment_Date: Date of payment received.
* Business Year: Year of the business transaction.
* Posting_Date: Date of posting the transaction.
* Due_Date: Date by which the payment is due.
* Payterm: Payment terms for the invoice.
* Invoice Currency: Currency in which the invoice is issued.
* Total Open Amount: Total amount of the invoice.
* USD_CURRENCY: Currency conversion rate to USD.
* Total Open Amount_USD: Total amount in USD.
* Invoice ID: Unique identifier for each invoice.
* Is Open: Indicates whether the invoice is open or closed.
* DUNNLEVEL: Dunn level of the invoice. Dunn level refers to the level of past-due status or aging of an invoice, indicating the severity or length of time the invoice remains unpaid, basically how many times the customer was contacted for payment and the status remained unchanged.
* Credit_limit: Credit limit assigned to the customer.
* Baseline_Date: Baseline date for the transaction.
* Region: Geographic region associated with the transaction.

In [9]:
receivables_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Business Code           50000 non-null  object        
 1   Customer Number         50000 non-null  object        
 2   Customer Name           50000 non-null  object        
 3   Payment_Date            50000 non-null  object        
 4   Business Year           50000 non-null  int64         
 5   Posting_Date            50000 non-null  datetime64[ns]
 6   Due_Date                50000 non-null  datetime64[ns]
 7   Payterm                 50000 non-null  int64         
 8   Invoice Currency        50000 non-null  object        
 9   Total Open Amount       50000 non-null  int64         
 10  USD_CURRENNCY           50000 non-null  object        
 11  Total Open Amount_USD   50000 non-null  float64       
 12  Customer Payment Terms  50000 non-null  object

There are no null values.
The is open column is for entries that do not have labels. They cannot be used for training and we have to remove them. 

In [10]:
receivables_df["Is Open"].value_counts()

Is Open
0    40000
1    10000
Name: count, dtype: int64

In [11]:
rslt_receivables_df = receivables_df[receivables_df["Is Open"] == 0]
rslt_receivables_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40000 entries, 0 to 49999
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Business Code           40000 non-null  object        
 1   Customer Number         40000 non-null  object        
 2   Customer Name           40000 non-null  object        
 3   Payment_Date            40000 non-null  object        
 4   Business Year           40000 non-null  int64         
 5   Posting_Date            40000 non-null  datetime64[ns]
 6   Due_Date                40000 non-null  datetime64[ns]
 7   Payterm                 40000 non-null  int64         
 8   Invoice Currency        40000 non-null  object        
 9   Total Open Amount       40000 non-null  int64         
 10  USD_CURRENNCY           40000 non-null  object        
 11  Total Open Amount_USD   40000 non-null  float64       
 12  Customer Payment Terms  40000 non-null  object     

The open transactions are removed. 
There are no null values. 
The payments column should be changed to a datetime type. 

In [12]:
rslt_receivables_df["Payment_Date"] = pd.to_datetime(rslt_receivables_df["Payment_Date"], format="%Y-%m-d")

<ipython-input-12-c48b322b6846>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_receivables_df["Payment_Date"] = pd.to_datetime(rslt_receivables_df["Payment_Date"], format="%Y-%m-d")


In [13]:
rslt_receivables_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40000 entries, 0 to 49999
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Business Code           40000 non-null  object        
 1   Customer Number         40000 non-null  object        
 2   Customer Name           40000 non-null  object        
 3   Payment_Date            40000 non-null  datetime64[ns]
 4   Business Year           40000 non-null  int64         
 5   Posting_Date            40000 non-null  datetime64[ns]
 6   Due_Date                40000 non-null  datetime64[ns]
 7   Payterm                 40000 non-null  int64         
 8   Invoice Currency        40000 non-null  object        
 9   Total Open Amount       40000 non-null  int64         
 10  USD_CURRENNCY           40000 non-null  object        
 11  Total Open Amount_USD   40000 non-null  float64       
 12  Customer Payment Terms  40000 non-null  object     

##### Suppliers

In [14]:
suppliers_df.head()

,Supplier ID,Supplier Name,Payment Terms,Vendor Type,Supplier Category
0,S-281,Roth-Sanchez,Net 60,Domestic,Raw Material
1,S-438,Peterson Inc,Net 60,Domestic,Raw Material
2,S-480,"Morton, Newman and Baker",Net 90,Domestic,Services
3,S-148,Evans Inc,Net 30,Domestic,Utility
4,S-8,Hart Ltd,Net 90,International,Taxes


In [15]:
suppliers_df.columns

Index(['Supplier ID', 'Supplier Name', 'Payment Terms', 'Vendor Type',
       'Supplier Category'],
      dtype='object')

These columns stand for:
* Supplier ID: Unique identifier for each supplier.
* Supplier Name: Name of the supplier.
* Payment Terms: Terms and conditions for supplier payments.
* Vendor Type: Type or category of the vendor/supplier.
* Supplier Category: Categorization of the supplier.

In [16]:
suppliers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Supplier ID        500 non-null    object
 1   Supplier Name      500 non-null    object
 2   Payment Terms      500 non-null    object
 3   Vendor Type        500 non-null    object
 4   Supplier Category  500 non-null    object
dtypes: object(5)
memory usage: 19.7+ KB


There are no null values. 

##### Payables

In [17]:
payables_df.head()

,Invoice Number,Posting Date,Invoice Date,Payment Date,Net Due Date (System Calculated Date),Supplier ID,Invoice Amount,Fiscal year,Overdue,Invoice Status,Spend Category,Total Outstanding amount,Late payment fees,Payterm_n,Vendor_Type
0,INV-5978675602067198,2019-04-12 00:00:00,2019-04-12 00:00:00,30-12-2019,2020-04-01 00:00:00,S-198,7147,2019-2020,0,Paid,Taxes,0,0,30,Domestic
1,INV-4385639898658799,16-03-2020,2020-12-03 00:00:00,NaN,16-04-2020,S-187,8575,2020-2021,75,Unpaid,Raw Material,8575,600,30,Domestic
2,INV-4586812911382721,26-12-2019,24-12-2019,2020-01-01 00:00:00,26-02-2020,S-300,6790,2019-2020,0,Paid,Raw Material,0,0,60,Domestic
3,INV-8091675473911407,14-01-2020,2020-10-01 00:00:00,NaN,14-02-2020,S-497,6575,2020-2021,137,Unpaid,Services,6575,822,30,Domestic
4,INV-9623721287362728,2019-08-12 00:00:00,2019-04-12 00:00:00,20-01-2020,2020-08-03 00:00:00,S-310,12635,2019-2020,0,Paid,Raw Material,0,0,90,Domestic


In [18]:
payables_df.columns

Index(['Invoice Number', 'Posting Date', 'Invoice Date', 'Payment Date',
       'Net Due Date (System Calculated Date)', 'Supplier ID',
       'Invoice Amount', 'Fiscal year', 'Overdue', 'Invoice Status',
       'Spend Category', 'Total Outstanding amount', 'Late payment fees',
       'Payterm_n', 'Vendor_Type'],
      dtype='object')

These columns stand for:
* Invoice Number: Unique identifier for each invoice.
* Posting Date: Date of posting the invoice.
* Invoice Date: Date of the invoice.
* Payment Date: Date of payment made.
* Net Due Date (System Calculated Date): Calculated date for net payment due.
* Supplier ID: Unique identifier for each supplier.
* Invoice Amount: Total amount of the invoice.
* Fiscal Year: Financial year associated with the invoice.
* Overdue: Indicates if the payment is overdue.
* Invoice Status: Status of the invoice (e.g., paid, outstanding).
* Spend Category: Categorization of the expenditure.
* Total Outstanding Amount: Total amount outstanding for the invoice.
* Late Payment Fees: Fees charged for late payments.
* Payterm_n: Payment terms for the invoice.
* Vendor Type: Type or category of the vendor/supplier.

In [19]:
payables_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Invoice Number                         20000 non-null  object
 1   Posting Date                           20000 non-null  object
 2   Invoice Date                           20000 non-null  object
 3   Payment Date                           13988 non-null  object
 4   Net Due Date (System Calculated Date)  20000 non-null  object
 5   Supplier ID                            20000 non-null  object
 6   Invoice Amount                         20000 non-null  int64 
 7   Fiscal year                            20000 non-null  object
 8   Overdue                                20000 non-null  int64 
 9   Invoice Status                         20000 non-null  object
 10  Spend Category                         20000 non-null  object
 11  Total Outstandi

Let us get the transactions that have already been paid. 

In [21]:
rslt_payables_df = payables_df[payables_df["Invoice Status"] == "Paid"]

In [22]:
rslt_payables_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13988 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Invoice Number                         13988 non-null  object
 1   Posting Date                           13988 non-null  object
 2   Invoice Date                           13988 non-null  object
 3   Payment Date                           13988 non-null  object
 4   Net Due Date (System Calculated Date)  13988 non-null  object
 5   Supplier ID                            13988 non-null  object
 6   Invoice Amount                         13988 non-null  int64 
 7   Fiscal year                            13988 non-null  object
 8   Overdue                                13988 non-null  int64 
 9   Invoice Status                         13988 non-null  object
 10  Spend Category                         13988 non-null  object
 11  Total Outstanding am

There are no null values. The unpaid invoices are removed.     
As seen above, the dates are in an object format, we want the in a datetime format.

In [25]:
# # converting date column from object to datetime
rslt_payables_df['Posting_Date'] = pd.to_datetime(rslt_payables_df['Posting Date'])
rslt_payables_df['Due_Date'] = pd.to_datetime(rslt_payables_df['Net Due Date (System Calculated Date)'])
rslt_payables_df['Invoice Date'] = pd.to_datetime(rslt_payables_df['Invoice Date'])
rslt_payables_df['Payment Date'] = pd.to_datetime(rslt_payables_df['Payment Date'])

<ipython-input-25-9e5ae0a06e58>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_payables_df['Posting_Date'] = pd.to_datetime(rslt_payables_df['Posting Date'])
<ipython-input-25-9e5ae0a06e58>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_payables_df['Due_Date'] = pd.to_datetime(rslt_payables_df['Net Due Date (System Calculated Date)'])
<ipython-input-25-9e5ae0a06e58>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [26]:
# Some basic data manipulation
rslt_payables_df['Invoice Amount'] = rslt_payables_df['Invoice Amount'].astype(float).round(1)
rslt_payables_df['Overdue'] = rslt_payables_df['Overdue'].astype(float).round(1)
rslt_payables_df['Total Outstanding amount'] = rslt_payables_df['Total Outstanding amount'].astype(float).round(1)
rslt_payables_df['Late payment fees'] = rslt_payables_df['Late payment fees'].astype(float).round(1)
rslt_payables_df['Payterm_n'] = rslt_payables_df['Payterm_n'].astype(int)

<ipython-input-26-0a1f1766176a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_payables_df['Invoice Amount'] = rslt_payables_df['Invoice Amount'].astype(float).round(1)
<ipython-input-26-0a1f1766176a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_payables_df['Overdue'] = rslt_payables_df['Overdue'].astype(float).round(1)
<ipython-input-26-0a1f1766176a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

# FEATURE ENGINEERING

### Receivables

We can check the difference between a customers due date and the payment date.      
The above can be used to classify if the customer was early, late or on time on his payments and add it as a separate column.  

In [23]:
# calculate the payments 
rslt_receivables_df["payment_flag"] = rslt_receivables_df["Due_Date"] - rslt_receivables_df["Payment_Date"]

# convert the dates to integers from datetime
rslt_receivables_df["payment_flag"] = pd.to_numeric(rslt_receivables_df["payment_flag"].dt.days, downcast="integer")

# create a payments flagname depending on the time paid
conditions = [
    (rslt_receivables_df["payment_flag"] == 0 ),
    (rslt_receivables_df["payment_flag"] < 0 ),
    (rslt_receivables_df["payment_flag"] > 0)]
choices = ['ontime', 'late', 'early']
rslt_receivables_df["payment_flagname"] = np.select(conditions, choices, default='NA')
rslt_receivables_df

<ipython-input-23-0f02e0d778a0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_receivables_df["payment_flag"] = rslt_receivables_df["Due_Date"] - rslt_receivables_df["Payment_Date"]
<ipython-input-23-0f02e0d778a0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_receivables_df["payment_flag"] = pd.to_numeric(rslt_receivables_df["payment_flag"].dt.days, downcast="integer")
<ipython-input-23-0f02e0d778a0>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,Business Code,Customer Number,Customer Name,Payment_Date,Business Year,Posting_Date,Due_Date,Payterm,Invoice Currency,Total Open Amount,...,Total Open Amount_USD,Customer Payment Terms,Invoice ID,Is Open,DUNNLEVEL,Credit_limit,Baseline_Date,Region,payment_flag,payment_flagname
0,U001,200769623,WAL-MAR corp,2020-02-11,2020,2020-01-26,2020-02-10,15,USD,54273,...,54273.2800,NAH4,1.930438e+09,0,0,50000,2020-01-23,WEST,-1,late
1,U001,200980828,BEN E,2019-08-08,2019,2019-07-22,2019-08-11,20,USD,79657,...,79656.6000,NAD1,1.929646e+09,0,2,50000,2019-07-20,MIDWEST,3,early
2,U001,200792734,MDV/ trust,2019-12-30,2019,2019-09-14,2019-09-29,15,USD,2254,...,2253.8600,NAA8,1.929874e+09,0,3,100000,2019-09-14,NORTHEAST,-92,late
4,U001,200769623,WAL-MAR foundation,2019-11-25,2019,2019-11-13,2019-11-28,15,USD,33133,...,33133.2900,NAH4,1.930148e+09,0,3,100000,2019-11-10,WEST,3,early
5,CA02,140106181,THE corporation,2019-12-04,2019,2019-09-20,2019-10-04,14,CAD,22226,...,16447.1216,CA10,2.960581e+09,0,3,100000,2019-09-16,SOUTHEAST,-61,late
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49994,U001,200762301,C&S WH trust,2019-07-25,2019,2019-07-10,2019-07-25,15,USD,84780,...,84780.4000,NAC6,1.929601e+09,0,2,100000,2019-07-06,MIDWEST,0,ontime
49996,U001,200769623,WAL-MAR co,2019-09-03,2019,2019-08-15,2019-08-30,15,USD,6767,...,6766.5400,NAH4,1.929744e+09,0,2,100000,2019-08-11,WEST,-4,late
49997,U001,200772595,SAFEW associates,2020-03-05,2020,2020-02-19,2020-03-05,15,USD,6121,...,6120.8600,NAA8,1.930537e+09,0,1,100000,2020-02-16,SOUTHEAST,0,ontime
49998,U001,200726979,BJ'S llc,2019-12-12,2019,2019-11-27,2019-12-12,15,USD,63,...,63.4800,NAA8,1.930199e+09,0,0,100000,2019-11-27,MIDWEST,0,ontime


In [27]:
# add an ID column
rslt_receivables_df['ID'] = range(1, len(rslt_receivables_df) + 1)

<ipython-input-27-f4598c336fa1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_receivables_df['ID'] = range(1, len(rslt_receivables_df) + 1)


We then group the two func

Customer Number
200769623    9581
200726979    1612
200762301    1225
200759878    1144
200794332     995
             ... 
200494781       1
100012836       1
200585100       1
100054351       1
200541916       1
Name: count, Length: 1056, dtype: int64

### Payables

We can check the difference between the payment date and the due date to and classify them as ontime, early and late. 

In [33]:
# Create a new column as the difference between the due date and the payables
rslt_payables_df['Payment_flag'] = rslt_payables_df['Due_Date'] - rslt_payables_df['Payment Date']
# converting days aging to integer
rslt_payables_df['Payment_flag'] = pd.to_numeric(rslt_payables_df['Payment_flag'].dt.days, downcast='integer')
# early,late ontime flag, this will be needed when we create customer level features
conditions = [
    (rslt_payables_df['Payment_flag'] == 0 ),
    (rslt_payables_df['Payment_flag'] < 0 ),
    (rslt_payables_df['Payment_flag'] > 0)]
choices = ['ontime', 'late', 'early']
rslt_payables_df['payment_flagname'] = np.select(conditions, choices, default='NA')
rslt_payables_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13988 entries, 0 to 19999
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   Invoice Number                         13988 non-null  object        
 1   Posting Date                           13988 non-null  object        
 2   Invoice Date                           13988 non-null  datetime64[ns]
 3   Payment Date                           13988 non-null  datetime64[ns]
 4   Net Due Date (System Calculated Date)  13988 non-null  object        
 5   Supplier ID                            13988 non-null  object        
 6   Invoice Amount                         13988 non-null  float64       
 7   Fiscal year                            13988 non-null  object        
 8   Overdue                                13988 non-null  float64       
 9   Invoice Status                         13988 non-null  object     

<ipython-input-33-93b7a38e446c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_payables_df['Payment_flag'] = rslt_payables_df['Due_Date'] - rslt_payables_df['Payment Date']
<ipython-input-33-93b7a38e446c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rslt_payables_df['Payment_flag'] = pd.to_numeric(rslt_payables_df['Payment_flag'].dt.days, downcast='integer')
<ipython-input-33-93b7a38e446c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [34]:
rslt_payables_df.head()

,Invoice Number,Posting Date,Invoice Date,Payment Date,Net Due Date (System Calculated Date),Supplier ID,Invoice Amount,Fiscal year,Overdue,Invoice Status,Spend Category,Total Outstanding amount,Late payment fees,Payterm_n,Vendor_Type,Posting_Date,Due_Date,Payment_flag,payment_flagname
0,INV-5978675602067198,2019-04-12 00:00:00,2019-04-12,2019-12-30,2020-04-01 00:00:00,S-198,7147.0,2019-2020,0.0,Paid,Taxes,0.0,0.0,30,Domestic,2019-04-12,2020-04-01,93,early
2,INV-4586812911382721,26-12-2019,2019-12-24,2020-01-01,26-02-2020,S-300,6790.0,2019-2020,0.0,Paid,Raw Material,0.0,0.0,60,Domestic,2019-12-26,2020-02-26,56,early
4,INV-9623721287362728,2019-08-12 00:00:00,2019-04-12,2020-01-20,2020-08-03 00:00:00,S-310,12635.0,2019-2020,0.0,Paid,Raw Material,0.0,0.0,90,Domestic,2019-08-12,2020-08-03,196,early
5,INV-6391763768436907,2019-11-03 00:00:00,2019-11-03,2019-03-24,2019-11-04 00:00:00,S-191,6808.0,2019-2020,0.0,Paid,Services,0.0,0.0,30,Domestic,2019-11-03,2019-11-04,225,early
7,INV-576255376736098,18-07-2019,2019-07-15,2019-10-24,18-08-2019,S-222,5834.0,2019-2020,67.0,Paid,Raw Material,0.0,402.0,30,Domestic,2019-07-18,2019-08-18,-67,late
